In [ ]:
# 10/03 Q test. bug. emailed author.
#     is Q having a unit of 1/V? give an 1/1000 to Q

In [2]:
from brian2 import *


In [47]:
start_scope()

defaultclock.dt = 0.025*ms

# Two-compartment structure
morpho = Soma(30*um)
morpho.spine = Cylinder(diameter=1*um, length=300*um, n=1)












Ne = 1

# some calcium-related vars, table 2
pHC = 4.0*umolar
aHC = 0.6*umolar
pHN = 4.
aHN = 3.
alphav = 1.
alphab = 5.
alphad = 1.
thetav = 2.*umolar
thetad = 2.6
thetab = 0.55
Cp = 5.
Cd = 4.
taup = 500*ms
taua = 5*ms
tauv = 10*ms
taud = 250*ms
taub = 40*ms
sigmav = -0.05*umolar
sigmad = -0.01
sigmab = -0.02
p = 0.3
d = 0.01
kd = -0.002
kp = -0.1

# some calcium-related vars, concentration
phi = 0.01*umolar/ms/(uamp*cm**-2)
beta = 0.083/ms
chi0 = 0.05*umolar
nbuff = 6.*umolar



# E cells stats
Cme = 1*ufarad/cm**2
Ele = -67*mV
ENae = 50*mV
Ek = -100*mV
gle = 0.1*msiemens/cm**2
gKe = 80*msiemens/cm**2

# AMPA E-E
syn_mod = 1.

gampae = 0.05*syn_mod*msiemens/cm**2

# gampae = 0.00*syn_mod/(Ne-1)*msiemens/cm**2

#ggabae = 0.03*syn_mod/(Ni)*msiemens/cm**2


# NMDA (excitatory)
g_syn_nmda = 0.3*syn_mod*msiemens/cm**2
g_cal_nmda_0 = 25*syn_mod*msiemens/cm**2
# g_NMDA_I = 0.258 * nS * 800. / N_E
V_cal = 140*mV
tausnmdar = 1*ms
tausnmdad = 100*ms
# Mg2 = 1.
Mg2 = 2.


# Ina params
gNae = 30*msiemens/cm**2
naatt = 1.
taumna = 0.05*ms
# tauhna = 0.5*ms   # dendrite
tauhna = 1.*ms  # soma



# Ina = gNae * mna**2 * hna * sna * (ENae-v)
# dendrite activation
# mnainf = 1/(1+exp(-(v+40*mV)/(3*mV))) : 1
# hnainf = 1/(1+exp((v+45*mV)/(3*mV))) : 1
# snainf = (1 + naatt*exp((v+60*mV)/(2*mV)))/(1+exp((v+45)/(3*mV))) : 1


# Ik params
gkdr = 14*msiemens/cm**2
taukdr = 3.5*ms
# dendrite activation
# mkdrinf = 1/(1+(-(v+49*mV)/(3*mV))) : 1

# Ia
ga = 7.5*msiemens/cm**2
F = 9.648e4 *coulomb
T = (273.16-273.15) * kelvin
degC = (300-273.15) *kelvin # in celsius
R = 8.315/kelvin *joule
# Q = F/coulomb/(R/(joule/degC)*(T+degC)/kelvin)/1000.  # /1000 - Volt to mV
Q = F/(R*(T+degC))*mV  # /1000 - Volt to mV
qt = (degC/kelvin - 24)/10.

# Imahp
gmahp = 25*msiemens/cm**2


# TEST
Isyn = 0*amp*meter**-2
Ileak = 0*amp*meter**-2
I_cal_nmda = 0*amp*meter**-2
Icoup = 0*amp*meter**-2
gNae = 0*msiemens/cm**2
# ga = 0*msiemens/cm**2
gmahp = 0*msiemens/cm**2
gkdr = 0*msiemens/cm**2


eqse = '''
dv/dt = ( ( Ileak +  Ina + Ik + Isyn + Icoup + I))/Cme : volt
#Ileak = gle * (Ele-v) : amp*meter**-2

I : amp*meter**-2

#Icoup : amp*meter**-2

#Ileak = gl*

Ina = gNae * mna**2 * hna * sna * (ENae-v) : amp*meter**-2
# soma activation
mnainf = 1/(1+exp(-(v+44*mV)/(3*mV))) : 1
hnainf = 1/(1+exp((v+49*mV)/(3.5*mV))) : 1
snainf = (1 + naatt*exp((v+60*mV)/(2*mV)))/(1+exp((v+45*mV)/(3*mV))) : 1
dmna/dt = (mnainf - mna)/taumna : 1
dhna/dt = (hnainf - hna)/tauhna : 1
dsna/dt = (snainf - sna)/tausna : 1
tausna = (0.00333*ms * exp(0.0024*(1/mV) * (v+60*mV) * Q))/(1 + exp(0.0012*(1/mV)*(v+60*mV)*Q)) : second

Ik = Ia + Ikdr + Imahp : amp*meter**-2

Ikdr = gkdr * mkdrinf**2 * (Ek - v) : amp*meter**-2
mkdrinf = 1/(1+exp(-(v+46.3*mV)/(3*mV))) : 1
dmkdr/dt = (mkdrinf - mkdr)/taukdr : 1

Ia = ga * ma * ha * (Ek-v) : amp*meter**-2
ksi = -1.5*(1/mV) - 1/(1+exp((v+40*mV)/(5*mV)))*(1/mV) : volt**-1
alphama = exp(1e-3 * ksi * (v-11*mV) * Q)/ms : Hz
betama = exp(0.00055 * ksi * (v-11*mV) * Q)/ms : Hz
alphaha = exp(0.003* (v+56*mV)/mV * Q)/ms : Hz
betaha = alphaha : Hz
dma/dt = alphama*(1-ma) - betama*ma : 1
dha/dt = alphaha*(1-ha) - betaha*ha : 1 
tauma = clip( betama /(0.05*(1/ms) * 1. * (1+alphama*ms)), 0.1,inf)*ms : second
tauha = clip(0.26*(1/mV)*(v+50*mV), 2, inf)*ms : second

Imahp = gmahp * mmahp * (Ek - v) : amp*meter**-2
alphamahp = 0.48*(1/ms)/(1+(0.18*mmolar)/chi*exp(-1.68*v/mV*Q)) : Hz
betamahp = 0.28*(1/ms)/(1+ chi/(0.011*mmolar*exp(-2*v*Q/mV))) : Hz
dmmahp/dt = alphamahp*(1-mmahp) - betamahp*mmahp : 1





dchi/dt = phi*I_cal_nmda - beta*(chi-chi0) -(beta/nbuff)*chi**2 : mmolar
dP/dt = ((10*(chi/pHC)**pHN / (1+(chi/pHC)**pHN))-Cp*A*P)/taup : 1
dA/dt = (((chi/aHC)**aHN/(1+(chi/aHC)**aHN))-A)/taua : 1
dV/dt = ( alphav/(1+exp((chi-thetav)/sigmav)) -V)/tauv : 1
dB/dt = ( alphab/(1+exp((A-thetab)/sigmab)) -B -Cd*V*B)/taub : 1
dD/dt = ( alphad/(1+exp((B-thetad)/sigmad)) -D)/taud : 1
'''

Ecells = NeuronGroup(Ne, eqse, threshold='v>0*mV',refractory='v > 0*mV',method='rk4')

# Me = StateMonitor(Ecells, 'v', record=True)



# ---------------- Initialization ---------------- #
Ecells.P = 0.3
Ecells.A = 0.5
Ecells.V = 0.01
Ecells.B = 2.5
Ecells.D = 0.01

Ecells.I = 1.6*uamp*cm**-2
Ecells.v = -60*mV
Ecells.chi = 0.5*umolar
# Ecells.g_cal_nmda = g_cal_nmda_0

g_cal_nmda = g_cal_nmda_0


# # ---------------- Synapses ---------------- #

# Coupling
# eqs_coup = '''
# Ileak = 1.125*(v_post - v_pre)/mV*uA
# '''

# C_coup = Synapses(Ecells, Ecells, model=eqs_coup, on_post = eqs_nmda_post,
#                  method='rk4')
# C_coup.connect(condition='i != j')




# mon = StateMonitor(Ecells, ['v','Ik','Ina'],record = True)
# mon = StateMonitor(Ecells, ['v','ksi','Ia','alphama','betama','alphaha','betaha'],record = True)
mon = StateMonitor(Ecells, ['v','Ina','Ik','Ia','Ikdr','Imahp'],record = True)



# mon = StateMonitor(Ecells, ['Ina','mna','hna','sna','tausna','snainf'],record = True)


# mon = StateMonitor(Ecells, ['v','Ik','Ina','ma','ha','alphama','betama'],record = True)


# mon_E = StateMonitor(Ecells, ['v','chi','P','V','A','D','B','I_NMDA_rec','I_cal_nmda','I_syn_nmda'],record = [0,1])
# mon_sp = SpikeMonitor(Ecells,record = True)



In [48]:
run(3*ms,report='stdout')

Starting simulation at t=0. s for a duration of 3. ms
0.003 (100%) simulated in < 1s


In [49]:
# (mon.v[0][0:200])
(mon.v.shape)

(1, 120)

In [53]:
# plot(mon.t,mon.v[0][:])
#      -mon.Ia[0][:])
# plot(mon.v[0][0:150]/mV)
# plot(mon.t,mon.v[0][0:150]/mV)
# mon.Ia[0][0:100]

In [67]:
# -1.5*(1/mV) - 1/(1+exp((-mon.v[0][:]+40*mV)/(5*mV)))*(1/mV)
# plot(mon.v[0][:])
mon.v[0][0:4]

array([-60.        , -59.98622109, -43.55243546,          nan]) * mvolt

In [6]:
# exp(1e-3 * mon.ksi[0] * (mon.v[0]-11*mV) * Q)
Q

0.43198536 * volt ** -1

In [131]:
1e-3 * mon.ksi[0] * (mon.v[0]-11*mV) * Q

array([182.93585598,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan])

In [64]:
# Q = F/coulomb/(R/(joule/degC)*(T+degC)/kelvin)
Q


0.020244132317562454